In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch

In [3]:
device = torch.device('cuda')
model = SentenceTransformer('sentence-transformers/sentence-t5-large').to(device)

In [4]:
df = pd.read_csv('../data/test/test_polyhope_english.csv')
texts = df['text'].tolist()
binary = df['binary'].tolist()
multiclass = df['multiclass'].tolist()

In [5]:
embeds = model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
# embeds = torch.tensor(embeds)
# torch.save(embeds, '../data/embeds/polyhope_test_english_embeds_t5_large.pt')

In [6]:
t = torch.load('../data/embeds/polyhope_train_english_embeds_t5_large.pt')

In [7]:
polyhope_binary_labels = {'Hope':1, 'Not Hope':0}
polyhope_inv_binary_labels = {v: k for k, v in polyhope_binary_labels.items()}

polyhope_multi_labels = {'Not Hope':0, 'Generalized Hope':1, 'Realistic Hope':2, 'Unrealistic Hope':3}
polyhope_inv_multi_labels = {v: k for k, v in polyhope_multi_labels.items()}

In [8]:
df_train = pd.read_csv('../data/train/train_polyhope_english.csv')
train_texts = df_train['text'].tolist()
train_bin = df_train['binary'].tolist()
train_multi = df_train['multiclass'].tolist()

In [9]:
from sklearn import svm

X = t.numpy()
Y = [polyhope_multi_labels[i] for i in train_multi]

clf = svm.SVC()
clf.fit(X, Y)

SVC()

In [12]:
df_val = pd.read_csv('../data/val/val_polyhope_english.csv')
val_texts = df_val['text'].tolist()
val_bin = df_val['binary'].tolist()
val_multi =df_val['multiclass'].tolist()

# for text, act in zip(val_texts[:10], val_bin[:10]):
#     print('Text: ', text)
#     print('Predicted ', polyhope_inv_binary_labels[clf.predict([model.encode(text)])[0]])
#     print('Actual : ', act)

In [14]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

pred = []
val_bin_c = [polyhope_multi_labels[i] for i in val_multi]
for text in tqdm(val_texts):
    pred.append(clf.predict([model.encode(text)])[0])

print(accuracy_score(val_bin_c, pred))
print(classification_report(val_bin_c, pred))


100%|████████████████████████████████████████████████████████████| 1032/1032 [00:56<00:00, 18.31it/s]

0.6744186046511628
              precision    recall  f1-score   support

           0       0.73      0.86      0.79       502
           1       0.61      0.65      0.63       300
           2       0.53      0.19      0.28       128
           3       0.58      0.43      0.49       102

    accuracy                           0.67      1032
   macro avg       0.61      0.53      0.55      1032
weighted avg       0.66      0.67      0.65      1032



In [18]:
pred = []
test_c = [polyhope_multi_labels[i] for i in multiclass]
for text in tqdm(texts):
    pred.append(clf.predict([model.encode(text)])[0])

print(accuracy_score(test_c, pred))
print(classification_report(test_c, pred))

100%|████████████████████████████████████████████████████████████| 1032/1032 [00:58<00:00, 17.56it/s]

0.7073643410852714
              precision    recall  f1-score   support

           0       0.73      0.89      0.80       491
           1       0.69      0.69      0.69       309
           2       0.65      0.24      0.35       124
           3       0.65      0.44      0.52       108

    accuracy                           0.71      1032
   macro avg       0.68      0.57      0.59      1032
weighted avg       0.70      0.71      0.69      1032

